In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from sys import path
import torch

In [2]:
from hsvi.pytorch import Hierarchy_SVI
from torch.distributions import Normal, Bernoulli

In [3]:
torch.autograd.set_detect_anomaly(True)

In [4]:
torch.random.manual_seed(0)

In [5]:
def softplus(x):
    return torch.log(torch.exp(x)+1.)

In [6]:
def simulator(N,M,T,r_per_p):
    reviews = pd.DataFrame(columns=['pid','rid','score'])
    pid = np.arange(N)
    reviews.pid = np.repeat(pid,r_per_p) # paper id of each review
    reviews.rid = np.random.choice(M,size=r_per_p*N) # reviewer id of each review
    #reviews.score = np.random.choice(np.arange(1,T+1),size=R)  #score 

    quality = np.random.normal(T/2+1,1., size=N)
    bias = np.random.normal(loc=0.,scale=0.5,size=[M,1])
    theta0 = np.array(np.arange(T)+1,ndmin=2,dtype=np.float32)
    theta0 = np.repeat(theta0,M,axis=0)

    delta = np.repeat(quality[reviews.pid].reshape(-1,1), T,axis=1) - (theta0 + np.repeat(bias,T,axis=1))[reviews.rid]

    y = (delta >= 0).astype(np.float32)

    reviews.score = y.sum(axis=1)
    return quality, bias, y, reviews
    

In [7]:
data_type = 'file' ## can be simulation or file

## Load and preprocess data

In [8]:
if data_type == 'simulation':
    N = 2000 # number of papers
    M = 100  # number of reviewers
    T = 4   # number of score levels
    r_per_p = 4 # number of reviews per submission
    true_quality, true_bias, y_data, reviews = simulator(N,M,T,r_per_p)
    id_map = pd.DataFrame(index=reviews.pid)
    id_map['id'] = reviews.pid.values
    y_data = torch.from_numpy(y_data.transpose())
else:
    data = pd.read_csv('./review_data.csv')
    ### form each entry as paper ID, reviewer ID, and score given by the reviewer to the paper ###
    reviews = pd.DataFrame(columns=['pid','rid','score'])
    reviews.pid = np.repeat(data.PaperID.values,2)
    for s in data.PaperID.values:
        reviews.loc[reviews.pid==s,'rid'] = data.loc[data.PaperID==s,['Rev1ID','Rev2ID']].values
        reviews.loc[reviews.pid==s,'score'] = data.loc[data.PaperID==s,['Rev1Score','Rev2Score']].values

    ### transform paper ID and reviewer ID to numbers ###
    reviews.pid = reviews.pid.map(lambda x: int(x[1:])-1)
    reviews.rid = reviews.rid.map(lambda x: int(x[1:])-1)
    
    ### generate mapping from pid to concecutive ID ###
    pid = data.PaperID.map(lambda x: int(x[1:])-1)
    id_map = pd.DataFrame(index=pid)
    id_map['id'] = data.index.values
    
    ### define hyper-parameters according to the data set ###
    N = data.shape[0] #number of papers
    R = reviews.shape[0] #number of reviews
    M = len(reviews.rid.unique()) #number of reviewers
    T = reviews.score.values.max() #number of score levels
    r_per_p = 2 # number of reviews per submission
    
    ### transform ovservations y ###
    y_data = np.ones((R,T))*np.arange(T)+1
    y_data = (y_data <= reviews.score.values.reshape(-1,1)).astype(dtype=np.float32)
    y_data = torch.from_numpy(y_data.transpose())


In [9]:
niter = 5000 # number of training iterations
local_iter = 1 # number of local iterations
theta_scale = .1

## Define the Reviewer-Bias IRT model

In [10]:
p_bias = Normal(0.,1.) # prior of bias
m = reviews.score.mean().astype(np.float32) # empirical mean of all score  
p_quality = Normal(loc=m*torch.ones([N]),scale=torch.ones([N])*2.) # prior mean set to empirical mean

theta0 = np.array(np.arange(T)+1,ndmin=2,dtype=np.float32)
theta0= torch.from_numpy(np.repeat(theta0,M,axis=0).transpose())

In [11]:
#b = np.array([reviews.loc[reviews.rid==r,'score'].mean()-np.mean([reviews.loc[reviews.pid==p, 'score'] for p in reviews.loc[reviews.rid==r,'pid']]) for r in np.arange(M)])
var_bias_loc = torch.tensor(torch.normal(torch.zeros([M]),torch.ones([M])*0.2),requires_grad=True) #
var_bias_scale = torch.ones([M],requires_grad=True)

var_quality_loc = torch.tensor(p_quality.sample(),requires_grad=True)
var_quality_scale = torch.ones([N],requires_grad=True)

/home/yu/gits/p36env/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/yu/gits/p36env/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [12]:
def generative_process(var_bias_loc,var_bias_scale,var_quality_loc,var_quality_scale):
    q_bias = Normal(loc=var_bias_loc,scale=softplus(var_bias_scale)) # posterior of bias
    q_quality = Normal(loc=var_quality_loc, scale=softplus(var_quality_scale)) # posterior of quality

    ### it is necessary to use rsample() here for enabling reparamterization trick ###
    theta_loc = theta0 + q_bias.rsample()
    qs = q_quality.rsample()
    
    if data_type == 'simulation':
        idx = reviews.pid.values
    else:
        idx = id_map.loc[reviews.pid.values,'id'].values
        
    quality = qs[torch.tensor(idx,dtype=torch.long)]
    
    roft = theta_loc[:,torch.tensor(reviews.rid.values,dtype=torch.long)]
    d_loc = quality-roft 
    d_scale = torch.ones_like(d_loc)*(np.sqrt((theta_scale**2)+(q_quality.scale.detach()[idx]**2)))
    d = Normal(d_loc, d_scale)
    y = Bernoulli(1.-d.cdf(torch.zeros_like(d_loc)))
    return y, q_bias, q_quality

## Define inference method for the model

In [13]:
inference = Hierarchy_SVI(var_dict={'reviewer':[var_bias_loc,var_bias_scale],'paper':[var_quality_loc,var_quality_scale]},learning_rate={'reviewer':0.001,'paper':0.001},train_size=N*r_per_p)

start init hsvi
reviewer KLqp
paper KLqp


## Training process

In [14]:
for _ in range(niter):  
    y, q_bias,q_quality = generative_process(var_bias_loc,var_bias_scale,var_quality_loc,var_quality_scale)
    inference.data = {'reviewer':{y:y_data},'paper':{y:y_data}}
    inference.latent_vars={'reviewer':{p_bias:q_bias},'paper':{p_quality:q_quality}}
    for __ in range(local_iter):
        inference.update(scope='paper',retain_graph=True)
        
    loss = inference.update(scope='reviewer',retain_graph=True)
    
    if (_+1)%500==0 or _==0:
        print(' loss {}'.format(loss))
    

/home/yu/gits/p36env/lib/python3.6/site-packages/torch/autograd/__init__.py:149: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10000). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:115.)
  allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag


 loss 1.5927212238311768
 loss 1.1935703754425049
 loss 0.7736486196517944
 loss 0.6971428990364075
 loss 0.6960224509239197
 loss 0.5786629915237427
 loss 0.5250568389892578
 loss 0.5316608548164368
 loss 0.5051803588867188
 loss 0.4727066159248352
 loss 0.47249045968055725


## Check results

In [15]:
### inferred bias of each reviewer ###
rbias = q_bias.loc.detach()
rb = pd.DataFrame(columns=['RVID','bias'])
rb.RVID = np.arange(M)
rb.bias = rbias
if data_type == 'simulation':
    rb['true_bias'] = true_bias
rb

,RVID,bias
0,0,-1.299779
1,1,-0.835793
2,2,-0.224752
3,3,-0.693915
4,4,-1.034683
5,5,-1.231208
6,6,-0.842081
7,7,-1.179815
8,8,-0.694147
9,9,-1.068105


In [16]:
### inferred quality of papers ###
quality=q_quality.loc.detach()

if data_type == 'simulation':
    qlt = pd.DataFrame(columns=['PID','quality','true_quality'])
    qlt.PID = np.arange(N)
    qlt.true_quality = true_quality
else:
    qlt = pd.DataFrame(columns=['PID','quality','avg_score'])
    qlt.PID = id_map.index.values
qlt.quality = quality
### compare the quality with average score ###
for i in qlt.PID:
    qlt.loc[qlt.PID==i,'avg_score'] = reviews.loc[reviews.pid==i,'score'].mean()

qlt.head()

,PID,quality,avg_score
0,0,2.712328,2.5
1,1,3.186133,3
2,2,3.215267,3
3,3,2.922653,2.5
4,5,2.484223,2.5
